This notebook will preprocess all of the data

In [12]:
# imports

import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from PIL import Image

In [19]:
# create label dict
label_dic = {'N': 0, 'O': 1, 'R': 2}
path = 'DATASET/'

paths = ["DATASET/TRAIN/N", "DATASET/TRAIN/O", "DATASET/TRAIN/R"]

train_N = os.listdir("DATASET/TRAIN/N")
train_O = os.listdir("DATASET/TRAIN/O")
train_R = os.listdir("DATASET/TRAIN/R")

test_N = os.listdir("DATASET/TEST/N")
test_O = os.listdir("DATASET/TEST/O")
test_R = os.listdir("DATASET/TEST/R")

print(train_N[:10])
print(test_N[:10])

# create list of training data
# training_data = []
# training_labels = []

# for label in os.listdir(path + "/TRAIN"):
#     for file in os.listdir(path + "/TRAIN/" + label):
#         training_data.append(file)
#         training_labels.append(label_dic[label])

# print(training_data[:10])
# print(training_labels[:10])

# create list of testing data
# testing_data = []
# testing_labels = []

# for label in os.listdir(path + "/TEST"):
#     for file in os.listdir(path + "/TEST/" + label):
#         testing_data.append(file)
#         testing_labels.append(label_dic[label])

# print(testing_data[:10])
# print(testing_labels[:10])

['R_1.jpg', 'R_10.jpg', 'R_100.jpg', 'R_1000.jpg', 'R_1001.jpg', 'R_1002.jpg', 'R_1003.jpg', 'R_1004.jpg', 'R_1005.jpg', 'R_1006.jpg']
['00000000.jpg', '00000001.jpg', '00000002.jpg', '00000003.jpg', '00000004.jpg', '00000006.jpg', '00000007.jpg', '00000008.jpg', '00000010.jpg', '00000011.jpg']


In [21]:
# preprocess each image into a vectorized form
train_data = []
train_labels = []

test_data = []
test_labels = []

for image in train_N:
    img = Image.open(paths[0] + "/" + image).convert('RGBA')
    arr = np.array(img)
    print(arr.shape)
    break


(160, 314, 4)
